**Cargamos las librerías necesarias**  
Utilizaremos sqlalchemy para la gestión de las conexiones de la BBDD.  
La utilización de un ORM en este caso solo nos aporta la gestión de conexiónes.
Se anexa otro Script en el que se crea la BBDD con un Modelo Relación como ejemplo. Esto aportaría además una capa agnostica con la BBDD.

**Configurar Logging**
Se configura el sistema de logging para que los mensajes se guarden en un archivo y se muestren en la consola.

In [1]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import logging
%run Functions.ipynb
# Configuramos los logs
# Crear la carpeta de logs si no existe
os.makedirs('logs', exist_ok=True)

# Configurar logging para guardar en archivo
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.FileHandler('logs/app.log'),
                        logging.StreamHandler()
                    ])
logger = logging.getLogger(__name__)

Para la conexión a una base de datos MySQL utilizando SQLAlchemy y la carga de variables de entorno desde un archivo .env. También se mencionarán las ventajas, desventajas y posibles mejoras.  
**Estructura del Archivo .env**
El archivo .env debe contener las credenciales y parámetros de conexión a la base de datos MySQL. La estructura es la siguiente:  
DB_USER=root  
DB_PASSWORD=root  
DB_HOST=localhost  
DB_PORT=3306  
DB_NAME=transporte_publico  

**Cargar Variables de Entorno**  
Se utiliza la biblioteca python-dotenv para cargar las variables de entorno desde el archivo .env.

In [2]:
load_dotenv()

# Obtener los parámetros de la base de datos desde las variables de entorno
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

# Crear la cadena de conexión
connection_string = f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'

# Crear el engine
engine = create_engine(connection_string)

try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT DATABASE()"))
        for row in result:
            logger.info(f"Connected to database: {row[0]}")
except Exception as e:
    logger.error(f"Error connecting to the database: {e}")


2024-07-23 17:41:00,236 - __main__ - INFO - Connected to database: transporte_publico


**Motivos y Ventajas:**
- Seguridad: Almacenar las credenciales en un archivo .env evita exponer información sensible en el código fuente.
- Flexibilidad: Permite cambiar las credenciales y parámetros de conexión sin modificar el código.
- Mantenibilidad: Facilita la gestión de configuraciones para diferentes entornos (desarrollo, pruebas, producción).

**Crear la Cadena de Conexión y el Engine**  
Se crea la cadena de conexión usando los parámetros cargados y se inicializa el engine de SQLAlchemy.
**Verificar la Conexión**  
Se verifica la conexión ejecutando una consulta simple para asegurarse de que el script se ha conectado correctamente a la base de datos.
**Motivos y Ventajas:**  
- Validación: Ejecutar una consulta de verificación asegura que la conexión se ha establecido correctamente.
- Diagnóstico: Permite identificar rápidamente problemas de conexión.  
#### Ventajas y Desventajas  
**Ventajas**  
- Seguridad: Las credenciales no se exponen en el código fuente.
- Flexibilidad y Mantenibilidad: Facilita la gestión de configuraciones y la adaptación a diferentes entornos.
- Estandarización: Utilizar archivos .env es una práctica común y recomendada en el desarrollo de software.  

**Desventajas**  
- Dependencia en el Archivo .env: Si el archivo .env no está presente o es incorrecto, el script no funcionará.
- Complejidad Adicional: Requiere la instalación y uso de una librería adicional (python-dotenv).

**Leer el Archivo SQL**  
Se lee el archivo SQL que contiene las declaraciones para crear las tablas necesarias.  

**Ejecutar Declaraciones SQL**  
Se ejecutan las declaraciones SQL leídas del archivo para crear las tablas.

In [3]:
# Leer el archivo SQL
sql_file_path = '../SQL/crearBBDDTransportesMysql.sql'
try:
    with open(sql_file_path, 'r', encoding='utf-8') as file:
        create_tables_sql = file.read()
    logger.info(f"SQL file '{sql_file_path}' read successfully.")
except FileNotFoundError as e:
    logger.error(f"Error reading SQL file: {e}")
    raise

# Ejecutar cada declaración SQL por separado
with engine.connect() as connection:
    for statement in create_tables_sql.split(';'):
        statement = statement.strip()
        if statement:
            try:
                connection.execute(text(statement))
                logger.info(f"Executed SQL statement: {statement[:50]}...")
            except Exception as e:
                logger.error(f"Error executing SQL statement: {e}")
                raise

2024-07-23 17:41:00,252 - __main__ - INFO - SQL file '../SQL/crearBBDDTransportesMysql.sql' read successfully.
2024-07-23 17:41:00,261 - __main__ - INFO - Executed SQL statement: -- Crear la base de datos
CREATE DATABASE IF NOT E...
2024-07-23 17:41:00,264 - __main__ - INFO - Executed SQL statement: -- Usar la base de datos
USE transporte_publico...
2024-07-23 17:41:00,271 - __main__ - INFO - Executed SQL statement: DROP TABLE IF EXISTS hechos_transporte...
2024-07-23 17:41:00,276 - __main__ - INFO - Executed SQL statement: DROP TABLE IF EXISTS dim_dia...
2024-07-23 17:41:00,280 - __main__ - INFO - Executed SQL statement: DROP TABLE IF EXISTS dim_region...
2024-07-23 17:41:00,285 - __main__ - INFO - Executed SQL statement: DROP TABLE IF EXISTS dim_transporte...
2024-07-23 17:41:00,289 - __main__ - INFO - Executed SQL statement: DROP TABLE IF EXISTS dim_ruta...
2024-07-23 17:41:00,324 - __main__ - INFO - Executed SQL statement: -- Crear la tabla de rutas
CREATE TABLE IF NOT EXI...
2024-

**Cargar el Archivo CSV**  
Se carga el archivo CSV que contiene los datos a procesar.  

**Limpieza de Datos**  
Se limpian los datos eliminando filas con valores no disponibles y convirtiendo columnas a tipo numérico.  

In [4]:
# Cargar el archivo CSV
file_path = '../Data/Transporte_Publico_Espana.csv'
try:
    data = pd.read_csv(file_path)
    logger.info(f"CSV file '{file_path}' loaded successfully.")
except FileNotFoundError as e:
    logger.error(f"Error loading CSV file: {e}")
    raise

# Limpieza de datos
data = valores_faltantes(data)
data['Numero_Pasajeros'] = pd.to_numeric(data['Numero_Pasajeros'], errors='coerce')
data['Duracion_Viaje_Minutos'] = pd.to_numeric(data['Duracion_Viaje_Minutos'], errors='coerce')
data['Retraso_Minutos'] = pd.to_numeric(data['Retraso_Minutos'], errors='coerce')

# Crear un diccionario para almacenar los ID de las dimensiones
dim_ruta = {}
dim_transporte = {}
dim_region = {}
dim_dia = {}

# Función para obtener o crear una dimensión
def get_or_create_dimension(connection, table, column, value, dim_dict):
    if value not in dim_dict:
        try:
            result = connection.execute(text(f"INSERT INTO {table} ({column}) VALUES (:value)"), {'value': value})
            dim_dict[value] = result.lastrowid
            logger.info(f"Inserted {value} into {table}, ID: {result.lastrowid}")
        except Exception as e:
            logger.error(f"Error inserting {value} into {table}: {e}")
            raise
    return dim_dict[value]

2024-07-23 17:41:00,756 - __main__ - INFO - CSV file '../Data/Transporte_Publico_Espana.csv' loaded successfully.


**Insertar Datos en la Base de Datos**  
Se insertan los datos en las tablas de dimensiones y hechos, gestionando las transacciones de manera segura.  

**Cierre de Conexión**  
Se fuerza el cierre de la conexión al final del proceso.

In [5]:
# Insertar datos en las tablas de dimensiones y hechos
with engine.connect() as connection:
    connection.execute(text("START TRANSACTION"))
    try:
        for _, row in data.iterrows():
            id_ruta = get_or_create_dimension(connection, 'dim_ruta', 'ruta', row['Ruta'], dim_ruta)
            id_transporte = get_or_create_dimension(connection, 'dim_transporte', 'tipo_transporte', row['Tipo_Transporte'], dim_transporte)
            id_region = get_or_create_dimension(connection, 'dim_region', 'region', row['Region'], dim_region)
            id_dia = get_or_create_dimension(connection, 'dim_dia', 'dia_semana', row['Dia_Semana'], dim_dia)

            connection.execute(text("""
                INSERT INTO hechos_transporte (
                    id_ruta, id_transporte, id_region, id_dia, numero_pasajeros, duracion_viaje_minutos, retraso_minutos
                ) VALUES (
                    :id_ruta, :id_transporte, :id_region, :id_dia, :numero_pasajeros, :duracion_viaje_minutos, :retraso_minutos
                )
            """), {
                'id_ruta': id_ruta,
                'id_transporte': id_transporte,
                'id_region': id_region,
                'id_dia': id_dia,
                'numero_pasajeros': row['Numero_Pasajeros'],
                'duracion_viaje_minutos': row['Duracion_Viaje_Minutos'],
                'retraso_minutos': row['Retraso_Minutos']
            })
        connection.execute(text("COMMIT"))
        logger.info("Transaction committed successfully.")
    except Exception as e:
        connection.execute(text("ROLLBACK"))
        logger.error(f"Error during transaction, rolled back: {e}")

# Forzar el cierre de la conexión
engine.dispose()
logger.info("Conexión cerrada y recursos liberados.")

print("Conexión cerrada y recursos liberados.")

2024-07-23 17:41:00,890 - __main__ - INFO - Inserted Malaga - Sevilla into dim_ruta, ID: 1
2024-07-23 17:41:00,892 - __main__ - INFO - Inserted Tren into dim_transporte, ID: 1
2024-07-23 17:41:00,895 - __main__ - INFO - Inserted Barcelona into dim_region, ID: 1
2024-07-23 17:41:00,899 - __main__ - INFO - Inserted Sunday into dim_dia, ID: 1
2024-07-23 17:41:00,904 - __main__ - INFO - Inserted Madrid - Zaragoza into dim_ruta, ID: 2
2024-07-23 17:41:00,906 - __main__ - INFO - Inserted Autobús into dim_transporte, ID: 2
2024-07-23 17:41:00,908 - __main__ - INFO - Inserted Madrid into dim_region, ID: 2
2024-07-23 17:41:00,913 - __main__ - INFO - Inserted Sevilla - Madrid into dim_ruta, ID: 3
2024-07-23 17:41:00,915 - __main__ - INFO - Inserted Metro into dim_transporte, ID: 3
2024-07-23 17:41:00,918 - __main__ - INFO - Inserted Malaga into dim_region, ID: 3
2024-07-23 17:41:00,921 - __main__ - INFO - Inserted Madrid - Malaga into dim_ruta, ID: 4
2024-07-23 17:41:00,924 - __main__ - INFO - I

Conexión cerrada y recursos liberados.
